<a href="https://colab.research.google.com/github/itimes-digital/deep-learning-estudo/blob/main/analise_petr4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [7]:
treinamento = pd.read_csv('https://raw.githubusercontent.com/itimes-digital/deep-learning-estudo/main/dataset/petr4_treinamento_ex.csv')

In [8]:
treinamento.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0


In [9]:
treinamento.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1340,2018-05-21,26.100000,26.290001,24.940001,25.049999,25.000000,75971900.0
1341,2018-05-22,24.809999,25.290001,24.150000,24.709999,24.709999,88109900.0
1342,2018-05-23,24.230000,24.480000,23.270000,23.270000,23.270000,74622500.0
1343,2018-05-24,20.049999,20.750000,19.650000,20.080000,20.080000,240343800.0
1344,2018-05-25,20.650000,21.270000,19.660000,19.799999,19.799999,131188100.0


In [10]:
treinamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345 entries, 0 to 1344
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1345 non-null   object 
 1   Open       1342 non-null   float64
 2   High       1342 non-null   float64
 3   Low        1342 non-null   float64
 4   Close      1342 non-null   float64
 5   Adj Close  1342 non-null   float64
 6   Volume     1342 non-null   float64
dtypes: float64(6), object(1)
memory usage: 73.7+ KB


In [11]:
treinamento.dropna(inplace=True)

In [12]:
treinamento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1342 entries, 0 to 1344
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1342 non-null   object 
 1   Open       1342 non-null   float64
 2   High       1342 non-null   float64
 3   Low        1342 non-null   float64
 4   Close      1342 non-null   float64
 5   Adj Close  1342 non-null   float64
 6   Volume     1342 non-null   float64
dtypes: float64(6), object(1)
memory usage: 83.9+ KB


In [13]:
treinamento_open = treinamento.iloc[:, 1:2].values
treinamento_open

array([[19.99    ],
       [19.809999],
       [20.33    ],
       ...,
       [24.23    ],
       [20.049999],
       [20.65    ]])

In [14]:
normalizador = MinMaxScaler(feature_range=(0,1))
treinamento_open_normlizado = normalizador.fit_transform(treinamento_open)
treinamento_open_normlizado

array([[0.68207343],
       [0.67429801],
       [0.69676026],
       ...,
       [0.86522678],
       [0.68466518],
       [0.71058315]])

In [15]:
treinamento_open_normlizado.shape

(1342, 1)

In [16]:
previsores = []
preco_real = []
ultimos_dias = 180

for i in range(ultimos_dias, treinamento_open_normlizado.shape[0]):
  previsores.append(treinamento_open_normlizado[i - ultimos_dias: i,0])
  preco_real.append(treinamento_open_normlizado[i, 0])

In [17]:
len(previsores)

1162

In [18]:
len(preco_real)

1162

In [19]:
previsores, preco_real = np.array(previsores), np.array(preco_real)

In [20]:
previsores

array([[0.68207343, 0.67429801, 0.69676026, ..., 0.64319654, 0.64319654,
        0.64319654],
       [0.67429801, 0.69676026, 0.70323974, ..., 0.64319654, 0.64319654,
        0.62764579],
       [0.69676026, 0.70323974, 0.68725706, ..., 0.64319654, 0.62764579,
        0.63153348],
       ...,
       [0.45140389, 0.46738661, 0.47084233, ..., 0.89892009, 0.94600432,
        0.89028073],
       [0.46738661, 0.47084233, 0.46133909, ..., 0.94600432, 0.89028073,
        0.86522678],
       [0.47084233, 0.46133909, 0.46565875, ..., 0.89028073, 0.86522678,
        0.68466518]])

In [21]:
previsores.shape

(1162, 180)

In [22]:
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))
previsores.shape

(1162, 180, 1)

In [24]:
model = Sequential()

model.add(LSTM(units = 180, return_sequences = True, input_shape = (previsores.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units = 150, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 100))
model.add(Dropout(0.2))

model.add(Dense(units = 1, activation = 'linear'))

model.compile(optimizer = 'rmsprop', 
              loss = 'mean_squared_error',
              metrics = ['mean_absolute_error'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 180, 180)          131040    
_________________________________________________________________
dropout (Dropout)            (None, 180, 180)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 180, 150)          198600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 180, 150)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               100400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                

In [ ]:
history_ = model.fit(previsores, preco_real, epochs = 500, batch_size = 32)

In [ ]:
np.mean(history_.history['mean_absolute_error'])

In [ ]:
plt.figure(figsize = (15, 10))
plt.plot(history_.history['mean_absolute_error'], color = 'red', label = 'mean_absolute_error')
plt.plot(history_.history['mean_squared_error'], color = 'green', label = 'mean_squared_error')
plt.title('Loss vs Error')
plt.legend()
plt.show()